# OpenStreetMap Data 

## Chosen Area

For this case study I chose to look at the map of my favorite city in the world, **Shenzhen, China**.

Open Street Map data is available at [openstreetmap.org](https://www.openstreetmap.org)

The map for Shenzhen was downloaded from [mapzen.com](https://mapzen.com/data/metro-extracts/metro/shenzhen_china/)

In this project I will looking through the OpenStreetMap data for Shenzhen to investigate the data, and check to see if there are any inaccuracies or inconsistencies which could be improved upon.

## Problems Encountered in the Map

- Mix of engish and chinese language. Sometimes the entries include both English and Chinese and sometimes only English.

- Road names with inconsitent endings, for example, "Street", "street", "St.", "St". 

- Suspicious postal codes. Chinese postal codes are 6 digits long and contain all numbers. Additionally, the zip codes for Shenzhen begin with 518, like 518000. There were a significant number of postal codes in the map which did not follow this pattern, such as "DD109 754" and "DD124 1957". I have a suspicion of what these are but some further querying later on in the report should help to clarify what the problem is.

### Inconsistent Street Names

In order to rectify the problem of having inconsistent street names, I first used following functions from the audit.py script to identify what types of street names existed in the map:

```
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types
```
Then I used the following update_name function to programmatically update the abbreviated street names with the correct names.
```
def update_name(name, mapping):
    words = name.split()
    for i in range(len(words)):
        if words[i] in mapping:
            words[i] = mapping[words[i]]
    name = " ".join(words)
    return name
```
Next I checked to see if the names were correctly updated, here are some of the results:
- Hai yue Rd => Hai yue Road
- 北环大道 North Ring Ave => 北环大道 North Ring Avenue
- 坂雪岗大道 Bǎnxuě Gǎng Av => 坂雪岗大道 Bǎnxuě Gǎng Avenue
- 南山大道 Nanshan Dadao => 南山大道 Nanshan DaDao

### Sorting by zip code and city
```
sqlite> SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags 
   ...> UNION ALL SELECT * FROM ways_tags) tags WHERE tags.key='postcode'
   ...> GROUP BY tags.value ORDER BY count DESC LIMIT 10;
```
|Postal Code|count|
|-----|-----|
|DD109 754|29|
|518000|11|
|DD117137|7|
|518048|5|
|DD1241957|5|
|00852|4|
|518038|4|
|518067|4|
|518040|3|
|518053|3|
  
Many of the zip codes are these strange values which begin with DD. Next I will sort by city and see if there is anything strange there which might shed some light on the zip code situation.
```
SELECT tags.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags 
WHERE tags.key='city' GROUP BY tags.value ORDER BY count DESC LIMIT 10;
```
|City|count|
|-----|-----|
|"香港 Hong Kong"|334|
|"Sai Kung"|52|
|"广东省深圳市"|50|
|"深圳"|43|
|"Ta Kwu Ling"|41|
|Shenzhen|37|
|"元朗 Yuen Long"|34|
|"Kam Tin"|29|
|"深圳市 Shenzhen"|25|
|"Shap Pat Heung"|18|

This query has highlighted two additional problems with the data.
- There is data from both Shenzhen and Hong Kong. I guess this is due to the fact that the map boundary was drawn as a rectangle and since the two cities are so close to eachother it was bound to capture some or Hong Kong.
- There are many different spellings of Shenzhen. "广东省深圳市" translates to Guangdong Province, Shenzhen City. "深圳" are the characters for Shenzhen. "深圳市 Shenzhen", contains both the english and chinese spellings of the city name.
Further data wrangling could be done to remove the data from Hong Kong and to programmatically update the city names to a more cnosistent format as I previously did with the street names.

## Overview of the data

(I chose not to make a smaller sample of the data because the original file size was relatively small)

|File|Size|
|-----|-----|
|Shenzhen_china.osm|126.7 MB|
|nodes.csv|49.5 MB|
|nodes_tags.csv|1.5 MB|
|ways.csv|3.8 MB|
|ways_nodes.csv|17.1 MB|
|ways_tags.csv|5.5 MB|

### Number of Nodes
```
sqlite> SELECT COUNT(*) FROM nodes;
```
601991

### Number of Ways
```
sqlite> SELECT COUNT(*) FROM ways;
```
64942

### Number of Unique Users
```
sqlite> SELECT COUNT(DISTINCT(m.uid))          
   ...> FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) m;
```
642

## Additional Queries

### Top 10 amenities
```
SELECT value, COUNT(*) as num FROM nodes_tags 
WHERE key=‘amenity' 
GROUP BY value 
ORDER BY num DESC 
LIMIT 10;
```
|Amenity|Count|
|-----|-----|
|toilets|324|
|post_box|166|
|shelter|163|
|restaurant|154|
|parking|109|
|bank|101|
|bus_station|101|
|bicycle_parking|86|
|fuel|83|
|fast_food|64|

### Top 10 Cuisines
```
SELECT nodes_tags.value, COUNT(*) as num FROM nodes_tags 
JOIN (SELECT DISTINCT(id) FROM nodes_tags 
WHERE value='restaurant') r ON nodes_tags.id=r.id 
WHERE nodes_tags.key=‘cuisine' 
GROUP BY nodes_tags.value 
ORDER BY num DESC 
LIMIT 10;
```
|Cuisine|Count|
|-----|-----|
|chinese|32|
|local|4|
|burger|3|
|cantonese|3|
|korean|3|
|regional|3|
|vegetarian|3
|ice_cream|2|
|noodle|2|
|pizza|2|
- Who would have thought Chinese would be the top cuisine?


### Top 5 Fast Food Restaurants
```
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value = 'fast_food') ff
ON nodes_tags.id=ff.id
WHERE nodes_tags.key='name'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
```
|Restaurant|Count|
|-----|-----|
|"McDonald's"|19|
|KFC|9|
|"大家樂 Café de Coral"|3|
|"Burger King"|2|
|McDonalds|2|

- It is not surprising that McDonalds wold be the top fast food restuarant in any city in the world, but it turns out that there are multiple spellings of McDonalds in this map. 
- I now want to check how many total McDonalds restaurants there are in the city, including variations in spelling.
```
SELECT nodes_tags.value, COUNT(*) FROM nodes_tags
WHERE value LIKE '%McDonald%';
```
**"McDonald's",36**

I am also curious how many Starbucks there are.
```
SELECT nodes_tags.value, COUNT(*) FROM nodes_tags
WHERE value LIKE '%Starbucks%';
```
**"Starbucks Coffee”,10**

### Top 3 Religions
```
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') rel
ON nodes_tags.id=rel.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 3;
```
|Religion|Count|
|-----|-----|
|christian|11|
|buddhist|9|
|taoist|8|

## Other Ideas About the Dataset

In order to get deeper in to the data I would need to do some additional data wrangling, including:
- Translating the entries in the dataset into a common language. Right now there are entries in English, Chinese characters, and Chinese Pinyin, which makes it difficult to get clear answers from any queries on the data. This of course would be a very difficult and time consuming process and would require help from someone with stronger chinese language abilities.
- Separating the data from Hong Kong and Shenzhen so that each city can be investigated separately and a clearer picture can be painted for each city individually. This could be done by identifiying which zip codes are from Hong Kong and eliminating all entries in the dataset associated with those zip codes.

## Conclusion

It is obvious that there were several problems with the portion of the map chosen for this project. The number of nodes and ways was significantly smaller than other US cities with smaller populations than Shenzhen meaning that there are fewer contributions to OpenStreetMap in Shenzhen. There were also many inconsistencies present in the form of language, abbreviations/misspellings, and entries which were actually from Hong Kong. Still, with the present data, in the condition that it was provided in, there was enough to learn some interesting patterns and it provided me with a great opportunity to practice wrangling and querying data about my favorite city in the world. 